In [6]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [7]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch03SparkToolSet").getOrCreate()

In [8]:
sc = spark.sparkContext
sc._jsc.sc().uiWebUrl().get()

'http://gw02.itversity.com:4053'

In [9]:
for x in sc._conf.getAll():
    if 'PROXY' in x[0]:
        print(x[1])
print(spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES'))
print(spark.conf.get('spark.driver.appUIAddress'))

rm01.itversity.com
http://rm01.itversity.com:19288/proxy/application_1533622723243_6152
http://rm01.itversity.com:19288/proxy/application_1533622723243_6152
http://gw02.itversity.com:4053


In [10]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/spark-guide/data/retail-data/by-day/*.csv")

AnalysisException: 'Path does not exist: hdfs://nn01.itversity.com:8020/user/kranthidr/spark-guide/data/retail-data/by-day/*.csv;'

In [ ]:
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [ ]:
staticDataFrame.printSchema()

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [ ]:
staticDataFrame.show()

In [ ]:
sc.version

In [ ]:
# COMMAND ----------
from pyspark.sql.functions import window, column, desc, col
staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .show(5)

In [ ]:
# COMMAND ----------
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/user/kranthidr/spark-guide/data/retail-data/by-day/*.csv")

In [ ]:
# COMMAND ----------
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [ ]:
# COMMAND ----------
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [ ]:
# COMMAND ----------

spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(20)

In [ ]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `window` DESC
  """)\
  .show(20)

In [ ]:
purchaseByCustomerPerHour.writeStream\
    .format("console")\
    .queryName("customer_purchases_2")\
    .outputMode("complete")\
    .start()

In [ ]:
# COMMAND ----------

from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)

In [ ]:
preppedDataFrame.show()

In [ ]:
# COMMAND ----------

trainDataFrame = preppedDataFrame\
  .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
  .where("InvoiceDate >= '2011-07-01'")

In [ ]:
# COMMAND ----------
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")

In [ ]:
# COMMAND ----------

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")

In [ ]:
# COMMAND ----------

from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")

In [ ]:
# COMMAND ----------

from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])

In [ ]:
# COMMAND ----------

fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [ ]:
# COMMAND ----------

transformedTraining = fittedPipeline.transform(trainDataFrame)

In [ ]:
transformedTraining.cache()

In [ ]:
# COMMAND ----------

from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
  .setK(20)\
  .setSeed(1)

In [ ]:
# COMMAND ----------

kmModel = kmeans.fit(transformedTraining)

In [ ]:
# COMMAND ----------

transformedTest = fittedPipeline.transform(testDataFrame)

In [ ]:
transformedTest.printSchema()

In [ ]:
kmModel.computeCost(transformedTest)

In [ ]:
kmModel.computeCost(transformedTraining)